In [ ]:
# S2 Sphere Geometry

In [ ]:
import src.params as params
params.manifold = 'S2'

In [ ]:
from src.manifold import *
from src.metric import *

from src.plotting import *
%matplotlib inline
plt.rcParams['figure.figsize'] = 7, 7

In [ ]:
# Observation and frame
ellipsoid.set_value(np.array([1,1,1]))
q0 = np.array([0.,0.]).astype(theano.config.floatX)
v0 = np.array([-1.,1.]).astype(theano.config.floatX)
v0 = 2*GramSchmidt(v0,q0)
p0 = gMflatf(q0,v0)

print("q = ", q0)
print("v = ", v0)
print("p = ", p0)

In [ ]:
# Curvature
from src.curvature import *
print("curvature = ", Rf(q0))

# Geodesics

### Geodesic equations

In [ ]:
# Geodesic equation
from src.geodesic_equation import *
# test geodesic
qsv = Expteqf(q0,v0)
%matplotlib notebook
newfig()
plotM(rotate=(30,-45))
plotx(qsv,v0,linewidth = 1.5, s=50)
plt.show()
#plt.savefig("/home/line/Dropbox/repos/theanodiffgeom-paper/paper/pictures/geoeqS2.pdf")

### Geodesics from Hamiltonian equations

In [ ]:
# Hamiltonian dynamics
from src.Hamiltonian import *
# test Hamiltionian and gradients
print(p0)
print(Hf(q0,p0))

# geodesic
qsv = Exptf(q0,p0).T
%matplotlib notebook
newfig()
plotM(rotate=(30,-45))
plotx(qsv,v0,linewidth=1.5,s=50)
plt.show()
#plt.savefig("/home/line/Dropbox/repos/theanodiffgeom-paper/paper/pictures/geodesicOnSphere.pdf")
(ts,qpsv) = Hamf(q0,p0)
psv = qpsv[:,1,:]
print("Energy: ",np.array([Hf(q,p) for (q,p) in zip(qsv,psv)]))

In [ ]:
# geodesic on R^2
#%matplotlib inline
plotR2x(qsv,v0)

# Geodesic on Frame bundle:

In [ ]:
# Observation
x0 = np.array([0.,0.]).astype(theano.config.floatX)

# Frame
v0 = np.array([[1.,-1.],[-1.,-1.]]).astype(theano.config.floatX)
ui0 = GramSchmidt(v0,x0) #sp.linalg.orth(v0) # Gram-Schmidt
#ui0 = np.linalg.solve(sp.linalg.sqrtm(gMf(x0)),v0)
q0 = np.hstack([x0,ui0.flatten()]).astype(theano.config.floatX)
print("q = ", q0)

# Moment vector:
xi0 = 3*np.array([1,-1]).astype(theano.config.floatX)
xia = 6*np.array([1,0,1,0]).astype(theano.config.floatX)

p0 = np.hstack([xi0,xia]).astype(theano.config.floatX)
print("p = ", p0)

In [ ]:
# Hamiltonian dynamics
from src.FM import *
# test Hamiltionian and gradients
print(p0)
print(Hfmf(q0,p0))

# geodesic
qsv = Exptfmf(q0,p0).T
#%matplotlib notebook
newfig()
plotM(rotate=(30,-20))
plotFMx(qsv,N_vec=5,linewidth=1.5,s=50)
plt.show()
#plt.savefig("/home/line/Dropbox/repos/theanodiffgeom-paper/paper/pictures/geodesicOnFMwspin.pdf")
(ts,qpsv) = Hamfmf(q0,p0)
psv = qpsv[:,1,:]
print("Energy: ",np.array([Hfmf(q,p) for (q,p) in zip(qsv,psv)]))

In [ ]:
#%matplotlib inline
plotR2FMx(qsv,N_vec=5)

# Curvature on FM

In [ ]:
import matplotlib.cm as cm
def plotCur2(q,v0,v1,color='b'):

        x = q[0:d.eval()]
        ui = q[d.eval():].reshape((d.eval(),2))
        
        xq = x
        if x.shape[0] < 3: # map to S2
            x = Ff(x)
         
        ax = plt.gca(projection='3d')
        ax.scatter(x[0],x[1],x[2],color=color)

        # Frame along curve:
        curm = np.tensordot(np.tensordot(R_uif(xq,ui), v0, axes = [0,0]), v1, axes = [0,0])
        colors = cm.rainbow(np.linspace(0, 1, d.eval()))
        for j in range(d.eval()):
            JFgammai = JFf(xq)
            uiq = np.dot(JFgammai,ui[j,:])
            curV = np.dot(JFgammai,curm[j,:])
            ax.quiver(x[0],x[1],x[2],uiq[0],uiq[1],uiq[2], pivot='tail',
                      arrow_length_ratio = 0.15, linewidths=1.5,
                      color=colors[j],normalize=True,length=1.5*np.linalg.norm(uiq)/2)
            #end_Hvecq = (x + uiq/2)
            ax.quiver(x[0],x[1],x[2],
                      curV[0],curV[1],curV[2], pivot='tail',
                      arrow_length_ratio = 0.15, linewidths=2,
                      color=colors[j],normalize=True,length=np.linalg.norm(uiq)/2)

In [ ]:
# Hamiltonian dynamics
from src.curvature import *

# Curvature
cur = R_uif(x0,ui0)

v0 = np.array([1,0])
v1 = np.array([0,1])
curm = np.tensordot(np.tensordot(cur, ui0[:,0], axes = [0,0]), ui0[:,1], axes = [0,0])
print("curvature = ", curm)

# Plot of curvature:
#%matplotlib notebook
newfig()
plotM(rotate=(50,-40))
plotCur2(q0,ui0[0,:],ui0[1,:])
plt.show()
#plt.savefig("/home/line/Dropbox/repos/theanodiffgeom-paper/paper/pictures/Curvature.pdf")

# Most Probable Pahts

In [ ]:
# Hamilhttp://localhost:8888/notebooks/S2.ipynb#tonian dynamics
from src.FM import *

x1 = np.array([0,0])
x2 = np.array([0.5,-0.5])

v0 = np.array([[1./3,0.],[0.,1./3]]).astype(theano.config.floatX)
ui0 = GramSchmidt(v0,x1)
q1 = np.hstack([x1,ui0.flatten()]).astype(theano.config.floatX)
print("q = ", q1)

xi0 = np.array([1,0]).astype(theano.config.floatX)
xia = np.array([0,0,0,0]).astype(theano.config.floatX)

p0 = np.hstack([xi0,xia]).astype(theano.config.floatX)
print("p = ",p0)

In [ ]:
# Tangent vector giving the MPP:
Hor_v = Logfm(q1,x2)

# MPP from q1 to x2:
qsv = Exptfmf(q1,gMflatf(q1,Hor_v)).T

#%matplotlib notebook
#%matplotlib inline
newfig()
ax = plt.gca(projection='3d')
plotM(rotate=(30,-20))
plotFMx(qsv,N_vec=5,linewidth=1.5,s=50)
xq = Ff(x2)
ax.scatter(xq[0],xq[1],xq[2],color='r')
plt.show()
#plt.savefig("/home/line/Dropbox/repos/theanodiffgeom-paper/paper/pictures/MPP.pdf")

# Parallel Transport

In [ ]:
v0 = np.array([-1./2,-1./2])
v0 = v0/np.linalg.norm(v0)
t = np.linspace(0,1,n_steps.get_value())
gamma0 = np.vstack([t**2,-np.sin(t)]).T
dgamma0 = np.vstack([2*t,np.cos(t)]).T

# MAKE STOCHASTIC DEVELOPMENT OF GAMMA!!!!!
from src.Stochastic_Development import *

x0 = np.array([0,0])
ui0 = np.array([[1./3,0.],[0.,1./3]]).astype(theano.config.floatX)
ui0 = GramSchmidt(ui0,x0)
start = time.time()
qsv = devf(q0,dgamma0)
diff = time.time() - start
print("Time for development = ", diff)

In [2]:
# Parallel transport
from src.parallel_transport import *
# test parallel transport
vt = partransf(v0,gamma0,dgamma0)
# plot of result
%matplotlib notebook
newfig()
plotM(rotate=(30,-45))
plotpar(gamma0,vt,v0)
plt.show()
#plt.savefig("/home/line/Dropbox/repos/theanodiffgeom-paper/paper/pictures/partrans.pdf")

NameError: name 'qsv' is not defined

# Development and Stochastic Development

### Development

In [ ]:
n_steps.set_value(100)
# Observation and frame
x0 = np.array([0.1,0.1]).astype(theano.config.floatX)

# Frame
v0 = np.array([[1,-1],[0,-1]]).astype(theano.config.floatX)
ui0 = 2*GramSchmidt(v0,x0)#sp.linalg.orth(v0) # Gramm-Schmidt!!

Rot = np.array([[np.cos(45),np.sin(45)],[-np.sin(45),np.cos(45)]])
ui0 = np.dot(Rot,ui0)
q0 = np.hstack([x0.flatten(),ui0.flatten()]).astype(theano.config.floatX)
print("q = ", q0)

# Curve in R^2:
t = np.linspace(0,10,n_steps.get_value()+1)
gamma0 = np.vstack([20*np.sin(t), t**2 + 2*t]).T
dgamma0 = np.diff(gamma0, axis = 0)
print("dgamma.shape =", dgamma0.shape)

In [ ]:
%matplotlib inline
plt.plot(gamma0[:,0],gamma0[:,1])
plt.plot(gamma0[0,0],gamma0[0,1], 'ro')
plt.plot(gamma0[-1,0],gamma0[-1,1],'go')
plt.axis((-21,21,-11,131))

In [ ]:
# Development dynamics
from src.Stochastic_Development import *
# test deterministic development
qsv = devf(q0,dgamma0)
#%matplotlib notebook
newfig()
plotM(rotate=(30,-45))
plotFMx(qsv,linewidth=1.5,s=50)
plt.show()
#plt.savefig("/home/line/Dropbox/repos/theanodiffgeom-paper/paper/pictures/development.pdf")

### Stochastic Development

In [ ]:
n_steps.set_value(1000)
x0 = np.array([0.,0.]).astype(theano.config.floatX)

v0 = np.array([[1.,-1],[1,1]]).astype(theano.config.floatX)
ui0 = GramSchmidt(v0,x0)#sp.linalg.orth(v0) # Gramm-Schmidt!!
print("x = ", x0)
print("ui = ", ui0)

q0 = np.hstack([x0.flatten(),ui0.flatten()]).astype(theano.config.floatX)
print("q = ", q0)

dWt0 = np.random.normal(0, np.sqrt(dt.eval()), (n_steps.get_value(),2))
drift0 = np.array([0.5,0.5])

In [ ]:
# test stochastic development
#%matplotlib notebook
qsv = stoc_devf(q0,dWt0,drift0)
newfig()
plotM(rotate=(30,-45))
plotFMx(qsv,linewidth=1.5,s=50)
plt.show()
#plt.savefig("/home/line/Dropbox/repos/theanodiffgeom-paper/paper/pictures/stocdev.pdf")

In [ ]:
%matplotlib inline
Wt0 = np.cumsum(dWt0,axis=0)
plt.plot(Wt0[:,0],Wt0[:,1],'b-')
plt.plot(0,0,'go',markersize=5.)
plt.plot(Wt0[-1,0],Wt0[-1,1],'ro')
#plt.savefig("/home/line/Dropbox/repos/theanodiffgeom-paper/paper/pictures/StocP.pdf")

## Brownian Motions

In [ ]:
# Brownian Motion dynamics
from src.Brownian_Stochastic_Development import *
# test Simulation of Brownian Motion
#%matplotlib notebook
qsv = SD_brownian(q0)
newfig()
plotM()
plotFMx(qsv,)
plt.show()

In [ ]:
# Brownian motion on R^2:
dWt0 = np.random.normal(0, np.sqrt(dt.eval()),
                        (n_steps.get_value(),2))

# Brownian Motion on S^2:
#%matplotlib notebook
qsv = SD_brownian(q0,2,dWt0)
newfig()
plotM()
plotFMx(qsv)
plt.show()

In [ ]:
# Updated until this point!

# Horizontal Vector Fields

In [ ]:
n_steps.set_value(100)
# Observation and frame
x0 = np.array([0.5,0.5]).astype(theano.config.floatX)

# Frame
v0 = np.array([[1,1],[0,-1]]).astype(theano.config.floatX)
ui0 = GramSchmidt(v0,x0)#sp.linalg.orth(v0) # Gramm-Schmidt!!

Rot = np.array([[np.cos(45),np.sin(45)],[-np.sin(45),np.cos(45)]])
ui0 = np.dot(Rot,ui0)
q0 = np.hstack([x0.flatten(),ui0.flatten()]).astype(theano.config.floatX)
print("q = ", q0)

t = np.linspace(0,1,n_steps.get_value())
gamma0 = 2*np.vstack([t**2+1,-t**2-np.sin(t)]).T
dgamma0 = 2*np.vstack([2*t,-2*t+np.cos(t)]).T

In [ ]:
def plotHorix(q,dgamma,i0=0,color='b',color_intensity=1.,linewidth=3.,prevx=None,last=True):
        if len(q.shape)>1:
            for i in range(q.shape[0]):
                plotHorix(q[i],dgamma,i0=i,# if prevx is None else None,
                      color=color,
                      linewidth=linewidth if i==0 or i==q.shape[0]-1 else .8,
                      color_intensity=color_intensity if i==0 or i==q.shape[0]-1 else .7,
                      prevx=q[i-1] if i>0 else None,
                      last=i==(q.shape[0]-1))
            return

        x = q[0:d.eval()]
        ui = q[d.eval():].reshape((d.eval(),2))
        
        xq = x
        if x.shape[0] < 3: # map to S2
            x = Ff(x)
         
        ax = plt.gca(projection='3d')
        if prevx is None or last:
            ax.scatter(x[0],x[1],x[2],color=color)
        if prevx is not None:
            prevxx = prevx[0:d.eval()]
            if prevxx.shape[0] < 3:
                prevxx = Ff(prevxx)
            xx = np.stack((prevxx,x))
            ax.plot(xx[:,0],xx[:,1],xx[:,2],linewidth=linewidth,color=color)

        # Frame along curve:
        if i0 is 0:
            Hvec = Horif(xq,ui)
            for j in range(d.eval()):
                JFgammai = JFf(xq)
                uiq = np.dot(JFgammai,ui[j,:])
                Hvecq = np.dot(JFgammai,Hvec[j,:])
                ax.quiver(x[0],x[1],x[2],uiq[0],uiq[1],uiq[2], pivot='tail',
                          arrow_length_ratio = 0.15, linewidths=1,
                          color='black',normalize=True,length=np.linalg.norm(uiq))
                ax.quiver(x[0],x[1],x[2],Hvecq[0],Hvecq[1],Hvecq[2], pivot='tail',
                          arrow_length_ratio = 0.15,normalize=True,linewidths=3, length=0.5,
                          color='red')
                #for k in range(rank.eval()):
                v = np.dot(Hvec[(2*j+2):(2*j+4),:],dgamma)
                Hvecui = np.dot(JFgammai,v)
                #Hvecui = np.dot(JFgammai,Hvec[(2*(j+1)+k),:])
                end_Hvecq = (x + uiq)
                ax.quiver(end_Hvecq[0],end_Hvecq[1],end_Hvecq[2],
                          Hvecui[0],Hvecui[1],Hvecui[2], pivot='tail',linewidths=2.,
                          normalize=True,length=0.3)
            # velocity vector:
            dgamma_v = np.dot(JFgammai,dgamma)
            ax.quiver(x[0],x[1],x[2],dgamma_v[0],dgamma_v[1],dgamma_v[2], pivot='tail',
                      arrow_length_ratio = 0.15,normalize=True,linewidths=3, length=0.7,
                      color='green')
                

In [ ]:
# Development dynamics
from src.Stochastic_Development import *
# test deterministic development
qsv = devf(q0,dgamma0)
Hori_v = Horif(x0,ui0)
print("Horizontal vector fields = ", Hori_v)

%matplotlib notebook
newfig()
plotM(rotate=(40,-45))
dqsv = qsv[1,0:d.eval()] - qsv[0,0:d.eval()]
plotHorix(qsv,dqsv)
#plt.savefig("/home/line/Dropbox/repos/theanodiffgeom-paper/paper/pictures/Hori.pdf")

In [ ]:
print(Hori_v[d.eval():,:])
print(np.dot(Hori_v[d.eval():(d.eval()+2),:],dgamma0[0,:]))
print(np.dot(Hori_v[(d.eval()+2):(d.eval()+4),:],dgamma0[0,:]))

In [ ]:
print(DgMf(np.array([0.,0.])))